In [44]:
import numpy as np

# **Step 1: Implementing Gates, Activation Function and Loss Functions:**

## 1- Gates:

In [45]:
class Gate:
    def forward(self):
        raise NotImplementedError
    
    def backward(self):
        raise NotImplementedError
    
class AddGate(Gate):
    def forward(self, x, y):
        self.x = x
        self.y = y
        return x + y
    
    def backward(self, dz):
        dx = dz * np.ones_like(self.x)
        dy = dz * np.ones_like(self.y) 
        return dx, dy
    
class MultiplyGate(Gate):
    def forward(self, x, y):
        self.x = x
        self.y = y
        return x * y
    
    def backward(self, dz):
        dx = dz * self.y
        dy = dz * self.x
        
        return dx, dy
    
    

## 2- Activation Functions:

In [46]:
class linear(Gate):
    def forward(self, x):
        self.x = x
        return x
    
    def backward(self, dz):
        return dz

class relu(Gate):
    def forward(self, x):
        self.x = x
        return np.maximum(0, x)
    
    def backward(self, dz):
        return dz if self.x > 0 else 0

class sigmoid(Gate):
    def forward(self, x):
        self.x = x
        return 1 / (1 + np.exp(-np.array(x)))
    
    def backward(self, dz):
        return dz * self.forward(self.x)(1 - self.forward(self.x))
    

class softmax(Gate):
    def forward(self, x):
        self.x = x
        return np.exp(x) / np.sum(np.exp(x), axis = -1, keepdims = True)
    
    def backward(self, dz):
        return dz * self.forward(self.x) * (1 - self.forward(self.x))

class tanh(Gate):
    def forward(self, x):
        self.x = x
        return np.tanh(x)
    
    def backward(self, dz):
        return dz * (1 - np.tanh(self.x) * 2)

## 3- Loss Functions:

In [47]:
class binary_cross_entropy(Gate):
    def forward(self, y, y_hat):
        self.y = y
        self.y_hat = y_hat
        return - np.mean( y * np.log(y_hat) + (1 - y) * np.log(1 - y_hat))
    
    def backward(self, dz):
        return dz * (self.y_hat - self.y) / (self.y_hat * (1 - self.y_hat))
    
    
class l2_loss(Gate):
    def forward(self, y, y_hat):
        self.y = y
        self.y_hat = y_hat
        return 0.5 * np.mean(np.power(y - y_hat, 2))
    
    def backward(self, dz):
        return dz * (self.y_hat - self.y)

### Test cases:

In [49]:
x = 3
y = 4

print(f"x = {x}, y = {y}")

add_gate = AddGate()
output = add_gate.forward(x,y)
print("Add Gate = ", output)

multiply_gate = MultiplyGate()
output = multiply_gate.forward(x,y)
print("Multiply Gate = ", output)

relu_function = relu()
output = relu_function.forward([-1, 2, 5, 0, -10, 6])
print("-> Relu of [-1, 2, 5, 0, -10, 6] = ", output)

sigmoid_function = sigmoid()
output = sigmoid_function.forward([-1, 2, 5, 0, -10, 6])
print("-> Sigmoid of [-1, 2, 5, 0, -10, 6] = ", output)

softmax_function = softmax()
output = softmax_function.forward([-1, 2, 5, 0, -10, 6])
print("-> Softmax of [-1, 2, 5, 0, -10, 6] = ", output)

tanh_function = tanh()
output = tanh_function.forward([-1, 2, 5, 0, -10, 6])
print("-> Tanh of [-1, 2, 5, 0, -10, 6] = ", output)

bce_loss = binary_cross_entropy()
result = bce_loss.forward(1, 0.8)
print("--> BCE loss = ", result)

mse_loss = l2_loss()
result = mse_loss.forward(1, 0.8)
print("--> L2 loss = ", result)

x = 3, y = 4
Add Gate =  7
Multiply Gate =  12
-> Relu of [-1, 2, 5, 0, -10, 6] =  [0 2 5 0 0 6]
-> Sigmoid of [-1, 2, 5, 0, -10, 6] =  [2.68941421e-01 8.80797078e-01 9.93307149e-01 5.00000000e-01
 4.53978687e-05 9.97527377e-01]
-> Softmax of [-1, 2, 5, 0, -10, 6] =  [6.56225724e-04 1.31806460e-02 2.64740352e-01 1.78380646e-03
 8.09846881e-08 7.19638889e-01]
-> Tanh of [-1, 2, 5, 0, -10, 6] =  [-0.76159416  0.96402758  0.9999092   0.         -1.          0.99998771]
--> BCE loss =  0.2231435513142097
--> L2 loss =  0.01999999999999999
